In [2]:
from webscraping import *
from leagues import *
from fake_useragent import UserAgent # pip install fake_useragent

user_agent = UserAgent()
header = {'User-Agent': user_agent.random}
#amanha = datetime.strftime(datetime.today(),'%d-%m-%Y')
amanha = datetime.strftime(datetime.today() + timedelta(days=1),'%Y-%m-%d')
partidas_amanha = TinyDB(f'./Jogos_do_Dia/{amanha}.json')
warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={header["User-Agent"]}')
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.maximize_window()

url = 'https://www.flashscore.com/'
driver.get(url)
try:
    WebDriverWait(driver, 8).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')))
    button_cookies = driver.find_element(By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')
    button_cookies.click()
except:
    print("cookies already closed")

WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'.calendar__navigation--tomorrow')))
driver.execute_script("document.getElementsByClassName('otPlaceholder')[0].style.display='none';")
driver.find_element(By.CSS_SELECTOR,'.calendar__navigation--tomorrow').click()
sleep(5)

jogos = driver.find_elements(By.CSS_SELECTOR,'div.event__match--twoLine')
id_jogos =[]
for i in jogos:
    id_jogos.append(i.get_attribute("id")[4:])

print(driver.title)
for id_jogo in tqdm(id_jogos):
    procura_duplicado = Query()
    if partidas_amanha.search(procura_duplicado.Id == id_jogo) == []:
        driver.get(f'https://www.flashscore.com/match/{id_jogo}/#/match-summary/match-summary')
        country = driver.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
        league = driver.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country > a').text.split(' -')[0]
        if f'{country} - {league}' in leagues_list:
            jogo = {}
            
            get_main_header(id_jogo,jogo,driver)
            get_odds_ml_ft(id_jogo,jogo,driver)
            get_odds_ou_ft(id_jogo,jogo,driver)
            get_odds_btts_ft(id_jogo,jogo,driver)
            get_odds_cs_ft(id_jogo,jogo,driver)

            partidas_amanha.insert(jogo.copy())
            
driver.close()
df = pd.DataFrame(partidas_amanha.all())


EURO U21 2023, Football Live Scores, Latest Football Results | Flashscore.com


  0%|          | 0/71 [00:00<?, ?it/s]

In [6]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [7]:
df

,Id,Date,Time,League,Home,Away,Round,Odd_H,Odd_D,Odd_A,Odd_Over25,Odd_Under25,Odd_BTTS_Yes,Odd_BTTS_No,Odd_CS_1-0,Odd_CS_2-0,Odd_CS_2-1,Odd_CS_3-0,Odd_CS_3-1,Odd_CS_3-2,Odd_CS_0-0,Odd_CS_1-1,Odd_CS_2-2,Odd_CS_3-3,Odd_CS_4-4,Odd_CS_0-1,Odd_CS_0-2,Odd_CS_1-2,Odd_CS_0-3,Odd_CS_1-3,Odd_CS_2-3
0,dOorzaBk,29/06/2023,08:35,CHINA - SUPER LEAGUE,Beijing Guoan,Shanghai Port,ROUND 13,3.40,3.60,2.15,1.57,2.35,1.50,2.50,15.0,19.0,12.0,34.0,23.0,26.0,15.0,7.0,11.0,34.0,101.0,11.0,13.0,9.0,21.0,15.0,21.0
1,vmymZIQe,29/06/2023,08:35,CHINA - SUPER LEAGUE,Shandong Taishan,Changchun Yatai,ROUND 13,1.50,4.33,6.50,1.57,2.35,1.67,2.10,9.0,8.5,8.5,11.0,11.0,21.0,17.0,8.5,15.0,41.0,151.0,21.0,34.0,19.0,67.0,41.0,41.0
2,hjuiYxt2,29/06/2023,08:35,CHINA - SUPER LEAGUE,Shenzhen,Chengdu Rongcheng,ROUND 13,15.00,5.50,1.25,1.67,2.15,2.25,1.57,26.0,51.0,34.0,201.0,101.0,81.0,12.0,11.0,26.0,81.0,501.0,7.0,5.5,10.0,7.0,12.0,34.0
3,8d02OIef,29/06/2023,13:00,EGYPT - PREMIER LEAGUE,El Ismaily,Aswan SC,ROUND 32,1.97,2.93,4.81,2.91,1.41,2.41,1.54,5.5,8.5,9.0,20.0,21.0,41.0,6.0,6.0,19.0,126.0,201.0,8.5,18.0,14.0,56.0,41.0,76.0
4,xW46NxB0,29/06/2023,15:30,EGYPT - PREMIER LEAGUE,National Bank Egypt,Al Ahly,ROUND 32,6.42,4.32,1.49,2.01,1.80,2.06,1.73,15.0,36.0,20.0,126.0,71.0,71.0,10.0,7.5,18.0,101.0,201.0,6.0,6.5,7.5,10.0,12.0,26.0
5,n7R0VhCE,29/06/2023,19:00,SOUTH AMERICA - COPA LIBERTADORES,Boca Juniors (Arg),Monagas (Ven),GROUP STAGE,1.29,5.00,13.00,1.90,1.95,2.63,1.44,5.5,5.5,10.0,7.0,13.0,41.0,9.0,9.5,29.0,101.0,501.0,21.0,51.0,34.0,151.0,101.0,101.0
6,UVG5UCRK,29/06/2023,19:00,SOUTH AMERICA - COPA LIBERTADORES,Colo Colo (Chi),Pereira (Col),GROUP STAGE,1.62,3.75,6.50,2.05,1.80,1.95,1.80,6.5,7.0,8.5,12.0,13.0,29.0,9.5,7.0,17.0,51.0,351.0,15.0,29.0,19.0,67.0,41.0,51.0
7,zVAtmIwH,29/06/2023,21:00,SOUTH AMERICA - COPA LIBERTADORES,Barcelona SC (Ecu),Cerro Porteno (Par),GROUP STAGE,1.85,3.60,4.00,1.80,2.00,1.70,2.05,8.0,9.5,8.5,15.0,15.0,23.0,12.0,7.0,13.0,41.0,201.0,13.0,21.0,13.0,41.0,29.0,34.0
8,0G8u7ITd,29/06/2023,21:00,SOUTH AMERICA - COPA LIBERTADORES,Palmeiras (Bra),Bolivar (Bol),GROUP STAGE,1.14,7.50,23.00,1.50,2.50,2.50,1.50,8.0,6.0,11.0,6.0,12.0,41.0,15.0,13.0,34.0,101.0,501.0,41.0,101.0,51.0,401.0,151.0,126.0


In [8]:
df = df[['Date', 'Time', 'League', 'Home', 'Away', 'Round', 'Odd_H', 'Odd_D', 'Odd_A', 'Odd_Over25', 'Odd_Under25', 'Odd_BTTS_Yes', 'Odd_BTTS_No']]

df.columns= ['Date','Time', 'League', 'Home', 'Away', 'Round', 'FT_Odd_H','FT_Odd_D','FT_Odd_A','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']

df.to_csv(f'./Jogos_do_Dia/{amanha}_Jogos_do_Dia.csv',index=False)

print('encerrado')

encerrado
